In [1]:
import re

In [2]:
from subprocess import run
from tempfile import mkdtemp
from os import listdir, unlink, removedirs
from os.path import join as pjoin
from time import monotonic

In [3]:
def rm_rf(d):
    for f in listdir(d):
        try:
            unlink(pjoin(d, f))
        except (OSError, IOError):
            pass
    try:
        removedirs(d)
    except (OSError, IOError):
        pass

In [4]:
def parse_buf(b):
    rv = dict()
    rv['fov'] = [float(x) for x in re.search(r'Field size: ([0-9.]+) x ([0-9.]+)', b).groups()]
    tmp = re.search(r'Field center: \(RA,Dec\) = \(([0-9.-]+), ([0-9.-]+)\) deg.', b).groups()
    rv['field_center'] = dict(zip(['ra', 'dec'], [float(x) for x in tmp]))
    rv['arcsec_pp'] = float(re.search(r'pixel scale ([0-9.]+) arcsec/pix', b).group(1))
    rv['constellations'] = re.findall(r'[Tt]he constellation (.+)', b)
    rv['stars'] = re.findall(r'The star (.+)', b)
    rv['index'] = re.search(r'Field \d+: solved with index index-([a-z0-9-]+).\S+endian.fits.', b).group(1)
    return rv

In [23]:
def solve_image(img, **kwargs):
    '''
    wrapper for astronomy.net solve-field
    
    Pretty much any command line parameter to solve-field can be specified.
    long_kw_param=2 -> "--long-kw-param 2"
    L=True -> "-L"
    
    Of particular interest:
    
    - scale_low (--scale-low)
    - scale_high (--scale-high)
    - cpulimit
    - sigma
    - depth
    
    '''

    #lbound=None, hbound=None):
    wd = mkdtemp(prefix='astrometry_', dir='/tmp')
    args = ['solve-field', '--dir', wd, '--temp-dir', wd] # '--new-fits', 'none']

    for k,v in kwargs.items():
        k = k.replace('_', '-')
        d = '-' if len(k) == 1 else '--'
        if v is True:
            args.append(d+k)
        else:
            args.extend([d+k, str(v)])
    
    args.append(img)
    t1 = monotonic()
    rv = run(args, capture_output=True)
    t2 = monotonic()
    rm_rf(wd)
    rv = parse_buf(rv.stdout.decode())
    rv['solve_time'] = t2-t1
    return rv

In [7]:
rv = solve_image('/home/ckuethe/astrometry/qhy_8mm.png', scale_low=20)

In [8]:
rv

{'fov': [32.8267, 25.0112],
 'field_center': {'ra': 103.956439, 'dec': -21.775036},
 'arcsec_pp': 94.663,
 'constellations': ['Canis Major (CMa)', 'Lepus (Lep)', 'Puppis (Pup)'],
 'stars': ['γLep',
  'Runaway Star (μCol)',
  'ζLep',
  'δLep',
  'λCol',
  'ηLep',
  'σCol',
  '3Mon',
  '17Lep',
  'θLep',
  '19Lep',
  'Furud (ζCMa)',
  'Mirzam (βCMa)',
  'δCol',
  'λCMa',
  'ξ1CMa',
  'ξ2CMa',
  'ν1CMa',
  'ν2CMa',
  'ν3CMa',
  'Sirius (αCMa)',
  '10CMa',
  '11CMa',
  'κCMa',
  '15CMa',
  'θCMa',
  'ο1CMa',
  '17CMa',
  'πCMa',
  'μCMa',
  'ιCMa',
  'Adhara / Adara (εCMa)',
  'σCMa',
  'ο2CMa',
  'Muliphen (γCMa)',
  'Wezen (δCMa)',
  '26CMa',
  '27CMa',
  'ωCMa',
  '29CMa',
  'τCMa',
  'Aludra (ηCMa)',
  'αMon',
  '1Pup',
  '3Pup',
  '4Pup',
  '5Pup',
  'οPup',
  '6Pup',
  'ξPup',
  '9Pup',
  '10Pup',
  '11Pup',
  '12Pup',
  'ρPup'],
 'index': 'tycho2-16',
 'solve_time': 7.685709367000072}

In [ ]:
x = rv['fov'][0] * 0.5
y = rv['fov'][1] * 0.5


In [16]:
fz = !ls -r /home/ckuethe/satvis-20200320/sv/*png

In [24]:
for f in fz:
    try:
        rv = solve_image(f, scale_low=22, scale_high=35, cpulimit=30)
        print(f, round(rv['solve_time'],2), rv['fov'], rv['field_center'])
    except KeyboardInterrupt:
        raise
    except Exception:
        print(f, "FAIL!")
        pass

/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202429-001396.png 38.41 [32.59, 24.7397] {'ra': 256.08777, 'dec': 68.841775}
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202428-001395.png 21.09 [32.5738, 25.0446] {'ra': 256.092028, 'dec': 68.838166}
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202417-001394.png 8.61 [32.6503, 24.9272] {'ra': 256.055963, 'dec': 68.818202}
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202415-001393.png 9.85 [32.6884, 24.7835] {'ra': 256.064952, 'dec': 68.833944}
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202353-001392.png FAIL!
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202352-001391.png FAIL!
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202343-001390.png FAIL!
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202342-001389.png FAIL!
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202332-001388.png FAIL!
/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202331-001387.png FAIL!
/home/ckuethe/satvis-

In [18]:
len(fz)

27

In [ ]:
#/home/ckuethe/satvis-20200320/sv/oaCapture-20200320-201936-001371.png 26.9  [32.8876, 25.0245] {'ra': 287.311042, 'dec': 74.976341}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202417-001394.png 8.61  [32.6503, 24.9272] {'ra': 256.055963, 'dec': 68.818202}
# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202415-001393.png 9.85  [32.6884, 24.7835] {'ra': 256.064952, 'dec': 68.833944}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202023-001373.png 25.75 [32.8741, 24.9987] {'ra': 248.207334, 'dec': 84.532979}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202144-001377.png 47.22 [33.2748, 25.2655] {'ra': 224.122897, 'dec': 80.670921}
# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202145-001378.png 33.87 [32.7692, 24.9961] {'ra': 223.590499, 'dec': 80.663944}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202236-001382.png 16.43 [32.9729, 24.9915] {'ra': 172.144072, 'dec': 58.075132}
# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202235-001381.png 9.92  [32.9982, 24.9857] {'ra': 172.026187, 'dec': 58.023744}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202247-001384.png 14.86 [33.025,  25.0374] {'ra': 168.682801, 'dec': 70.140929}
# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202246-001383.png 12.48 [32.7838, 24.9829] {'ra': 168.700729, 'dec': 70.196889}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202226-001380.png 10.45 [32.8052, 24.9672] {'ra': 167.397056, 'dec': 38.290647}
# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202224-001379.png 10.69 [32.7796, 24.9864] {'ra': 167.383133, 'dec': 38.286455}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-202258-001386.png 9.96  [32.9514, 25.0092] {'ra': 131.660697, 'dec': 81.187798}

# /home/ckuethe/satvis-20200320/sv/oaCapture-20200320-201857-001370.png 15.97 [32.8686, 25.0182] {'ra':   9.885892, 'dec': 85.849623}
